# Optimizing over trained graph neural networks

This notebook explains how OMLT is used to optimize over trained graph neural networks (GNNs). We follow the below steps:

1.) A general definition of GNNs is provided. OMLT currently only supports GNN that fits our definition. 

2.) Introduce the `GNNLayer` class inside OMLT.

3.) Derive the big-M formulation for GNN layers.

4.) List operations that are implemented inside OMLT. OMLT can automatically encode a given GNN consists of these operations. 

5.) For customized GNNs, we give examples to illustrate how to transform it into OMLT. 

6.) Examples: one has fixed graph structure, another one has non-fixed graph structure. For each case, the output of the GNN is minimized.

**NOTE:** For simplicity, we skip the training process and just use random parameters for GNNs.


## Library Setup

This notebook assumes you have a working PyTorch environment to define a Dense NN. This Dense NN is then formulated in Pyomo using OMLT which therefore requires working Pyomo and OMLT installations.

The required Python libraries used in this notebook are as follows:

- `numpy`: used for transformation of parameters

- `torch`: the machine learning language used for neural networks

- `torch_geometric`: the machine learning language used for graph neural networks

- `pyomo`: the algebraic modeling language for Python, it is used to define the optimization model passed to the solver

- `onnx`: used to express trained neural network models

- `omlt`: the package this notebook demonstrates. OMLT can formulate machine learning (such as neural networks) within Pyomo

**NOTE:** This notebook also assumes you have a working MIP solver executable to solve optimization problems in Pyomo. The open-source solver CBC is called by default. 


## Definition of GNNs

We define a GNN with $L$ layers as follows:

\begin{equation*}
	\begin{aligned}
		GNN:\underbrace{\mathbb R^{d_0}\otimes\cdots\otimes\mathbb R^{d_0}}_{N \rm{times}}\to\underbrace{\mathbb R^{d_L}\otimes\cdots\otimes\mathbb R^{d_L}}_{N\ \rm{times}}
	\end{aligned}
\end{equation*}
    
where $V$ is the set of nodes of the input graph, $N=|V|$ is the number of nodes. 

Let $\mathbf{x}_v^{(0)} \in \mathbb{R}^{d_0}$ be the input features for node $v$. Then, the $l$-th layer ($l=1,2,\dots,L$) is defined by:

\begin{equation*}
	\begin{aligned}
		\mathbf{x}_v^{(l)}=\sigma\left(\sum\limits_{u\in\mathcal N(v)\cup\{v\}}\mathbf{w}_{u\to v}^{(l)}\mathbf{x}_u^{(l-1)}+\mathbf{b}_{v}^{(l)}\right),~\forall v\in V
	\end{aligned}
\end{equation*}

where $\mathcal N(v)$ is the set of all neighbors of $v$, $\sigma$ could be identity or any activation function.

*Dimensionality:* $\mathbf{x}_u^{(l-1)}\in\mathbb R^{d_{l-1}}, \mathbf{x}_v^{(l)},\mathbf{b}_v^{(l)}\in\mathbb R^{d_l}, \mathbf{w}_{u\to v}^{(l)}\in\mathbb R^{d_l}\times \mathbb R^{d_{l-1}}$.

## GNN Layers in OMLT

For optimization purposes, OMLT requires a given number of nodes $N$ in the input graph. Each GNN layer will be expanded as shown in follows.

Stack $\{\mathbf{x}_v^{(l)}\}_{v\in V}$ as a vector $\mathbf{X}^{(l)}\in \mathbb R^{Nd_l}$. Rewrite previous definition as:
\begin{equation*}
	\begin{aligned}
		\mathbf{X}^{(l)}=\sigma\left(\mathbf{W}^{(l)}\mathbf{X}^{(l-1)}+\mathbf{B}^{(l)}\right)
	\end{aligned}
\end{equation*}

One needs to provide $\mathbf{W}^{(l)}\in\mathbb R^{Nd_l\times Nd_{l-1}}, \mathbf{B}^{(l)}\in\mathbb R^{Nd_l}, N$ to define a `GNNLayer` in OMLT. 

**NOTE:** To keep consistency with other types of layers, all weights are transposed.

If the input graph structure is fixed, then weights $\mathbf{w}_{u\to v}^{(l)}$, biases $\mathbf{b}_{v}^{(l)}$, and links between layers determined by $\mathcal N(v)$ are all fixed after the GNN is trained. In this case, $\mathbf{W}^{(l)}$ is a sparse matrix with nonzero sub-matrices $\{\mathbf{w}_{u\to v}^{(l)}\}_{v\in V,u\in\mathcal N(v)\cup\{v\}}$ and $\mathbf{B}^{(l)}$ is the stack of $\{\mathbf{b}_v^{(l)}\}_{v\in V}$. The mixed-integer formulation of `GNNLayer` can be interpreted from two perspectives: (1) the same as a `DenseLayer`; (2) a simplified setting of `GNNLayer` with non-fixed input graph (introduced later).   

If the input graph structure is not fixed, then all weights $\mathbf{w}_{u\to v}^{(l)}$ and biases $\mathbf{b}_{v}^{(l)}$ are needed to build mixed-integer formulations. In this case, $\mathbf{B}^{(l)}$ is still the stack of $\{\mathbf{b}_v^{(l)}\}_{v\in V}$, while $\mathbf{W}^{(l)}$ is a dense matrix consists of $\{\mathbf{w}_{u\to v}^{(l)}\}_{u,v\in V}$.

## Formulation for GNN Layers

When the input graph structure is not fixed, elements in the adjacency matrix $A$ are decision variables. In this case, $\mathcal N(v)$ is not given anymore. Additionally, $\mathbf{w}_{u\to v}^{(l)},\mathbf{b}_v^{(l)}$ may contain the graph information, which makes them be variables. The formulation for GNN layers is built assuming that $\mathbf{w}_{u\to v}^{(l)},\mathbf{b}_v^{(l)}$ are fixed.

First, observe that the existence of edge $u\to v$ determines the contribution link from $\mathbf{x}_u^{(l-1)}$ to $\mathbf{x}_v^{(l)}$. Adding binary variables $A_{u,v}$ for all $u,v\in V$, we can formulate GNNs in a bilinear way:
\begin{equation*}
    \begin{aligned}
        \mathbf{x}_v^{(l)}=\sigma\left(\sum\limits_{u\in V}A_{u,v}\mathbf{w}_{u\to v}^{(l)}\mathbf{x}_u^{(l-1)}+\mathbf{b}_{v}^{(l)}\right), \forall v\in V
    \end{aligned}
\end{equation*}

This formulation involves quadratic constraints. To avoid them, instead of using binary variables to directly control the existence of contributions between nodes, we introduce introduces auxiliary variables $\mathbf{\bar x}_{u\to v}^{(l-1)}$ to represent the contribution from node $u$ to node $v$ in $l$-th layer:
\begin{equation*}
    \begin{aligned}
        \mathbf{x}_v^{(l)}=\sigma\left(\sum\limits_{u\in V}\mathbf{w}_{u\to v}^{(l)}\mathbf{\bar x}_{u\to v}^{(l-1)}+\mathbf{b}_{v}^{(l)}\right), \forall v\in V
    \end{aligned}
\end{equation*}
where
\begin{equation*}
    \begin{aligned}
        \mathbf{\bar x}_{u\to v}^{(l-1)}=\begin{cases}
            0, & A_{u,v}=0\\
            \mathbf{x}_u^{(l-1)}, & A_{u,v}=1
        \end{cases}
    \end{aligned}
\end{equation*}
Assume that each feature is bounded, then the definition of $\mathbf{\bar x}_{u\to v}^{(l-1)}$ could be reformulated using big-M:
\begin{equation*}
    \begin{aligned}
        \mathbf{x}_{u}^{(l-1)}-\mathbf{M}_{u}^{(l-1)}(1-A_{u,v})\le &~\mathbf{\bar x}_{u\to v}^{(l-1)}\le \mathbf{x}_{u}^{(l-1)}+\mathbf{M}_{u}^{(l-1)}(1-A_{u,v})\\
        -\mathbf{M}_{u}^{(l-1)}A_{u,v}\le &~\mathbf{\bar x}_{u\to v}^{(l-1)}\le \mathbf{M}_u^{(l-1)}A_{u,v}
    \end{aligned}
\end{equation*}
where $|\mathbf{x}_u^{(l-1)}|\le \mathbf{M}_u^{(l-1)}, A_{u,v}\in\{0,1\}$. By adding extra continuous variables and constraints, as well as utilizing the bounds for all features, the big-M formulation replaces the bi-linear constraints by linear constraints. OMLT uses this big-M formulation as the default (and only) formulation for GNN layers.

**NOTE:** When the input graph structure is fixed, we can fix $A$ to reduce the big-M formulation.

## Implemented GNN Operations in OMLT

The following operations from `torch_geometric` are implemented in OMLT:

- Convolutional Layers: `Linear`, `GCNConv`, `SAGEConv`
- Aggregation Operators: `sum`, `mean`
- Pooling Layers: `global_mean_pool`, `global_add_pool`
- Activation Functions: all activations supported in OMLT are compatible with these GNN operations.

**NOTE:** When the input graph is not fixed, there is no graph information. `GCNConv` layer and `mean` aggregation are not supported.

OMLT provides two functions `gnn_with_fixed_graph` and `gnn_with_non_fixed_graph` to encode GNNs with fixed/non-fixed input graph structure. Both functions only require (1) a sequential model from `torch_geometric`, and (2) number of nodes $N$ (and the adjacency matrix $A$ for fixed graph cases). The basic pipeline of both functions are:

1.) Transform each operation, e.g., linear layers and pooling layers will be transformed into `DenseLayer` (which is straightforward), GNN layers will be rewritten into `GNNLayer`, activation functions are identified and absorbed into corresponding layers.

2.) Define binary variables $A_{u,v}$ for adjacency matrix. We always assume $A$ is symmetric (i.e., $A_{u,v}=A_{v,u}$) and has non-zero diagonal elements (i.e., $A_{v,v}=1$). When $A$ is given for fixed graph cases, these variables will then be fixed.

3.) Build formulation. Currently, we only support `FullSpaceNNFormulations`. ReLU activation functions are encoded into linear constraints using a big-M formulation. For smooth activation functions (e.g., Sigmoid, LogSoftmax, Tanh), a smooth optimization solvers (such as Ipopt) is needed to handle nonlinear constraints.

## How to Transform Your Own GNN into OMLT

As mentioned before, any GNN that satisfies our GNN definition could be transformed into OMLT and then encoded using big-M formulation. Here we give two examples to show how to transform an outside GNN into OMLT.

The first example corresponds to fixed graph cases. Given a simple GNN consists of a GraphSAGE layer, an add pooling layer, and a dense layer with single output. Let the input and output features of the GraphSAGE layer are 2 and 3, respectively. 

The GraphSAGE layer is defined by:
\begin{equation*}
    \mathbf{x}_v^{(l)}=\sigma\left(\mathbf{w_1}^{(l)}\mathbf{x}_v^{(l-1)}+\mathbf{w_2}^{(l)}\sum\limits_{u\in\mathcal N(v)}\mathbf{x}_u^{(l-1)}+\mathbf{b}^{(l)}\right)
\end{equation*}
where a sum aggregation is used.

For the fixed graph structure, assume that it is a line graph with $N=3$ nodes, i.e., the adjacency matrix $A=\begin{pmatrix}1 & 1 & 0\\1 & 1 & 1\\ 0 & 1 & 1\end{pmatrix}$. Then the GraphSAGE layer could be rewritten as a `GNNLayer` with parameters:

   \begin{equation*}
        \mathbf{W}=\begin{pmatrix}
            \mathbf{w_1} & \mathbf{w_2} & \mathbf{0} \\
            \mathbf{w_2} & \mathbf{w_1} & \mathbf{w_2} \\
            \mathbf{0} & \mathbf{w_2} & \mathbf{w_1} \\
        \end{pmatrix},
        \mathbf{B}=\begin{pmatrix}
        \mathbf{b}\\\mathbf{b}\\\mathbf{b}
        \end{pmatrix}
    \end{equation*}
    
See below as a mapping between the given outside GNN and its corresponding GNN inside OMLT in form "layer type (in_channel, out_channel)":

\begin{equation*}
    \begin{aligned}
        \text{GraphSAGE(2, 3)}   &\Rightarrow \text{GNNLayer(6, 9)}\\
        \text{add pooling(9, 3)}  &\Rightarrow \text{DenseLayer(9, 3)}\\
        \text{dense(3, 1)}      &\Rightarrow \text{DenseLayer(3, 1)}
    \end{aligned}
\end{equation*}

The second example reuses the GNN architecture but no longer fixes the input graph structure. In such setting, all $\mathbf{w}_{u\to v}^{(l)},\mathbf{b}_v^{(l)}$ should be provided. Therefore, the `GNNLayer` is defined by:

\begin{equation*}
        \mathbf{W}=\begin{pmatrix}
            \mathbf{w_1} & \mathbf{w_2} & \mathbf{w_2} \\
            \mathbf{w_2} & \mathbf{w_1} & \mathbf{w_2} \\
            \mathbf{w_2} & \mathbf{w_2} & \mathbf{w_1} \\
        \end{pmatrix},
        \mathbf{B}=\begin{pmatrix}
        \mathbf{b}\\\mathbf{b}\\\mathbf{b}
        \end{pmatrix}
    \end{equation*}

## Example 1: Optimizing a GNN with Fixed Graph

Define a GCN in `torch_geometric` as follows:

In [2]:
import numpy as np
import torch
from torch.nn import Linear, ReLU
from torch_geometric.nn import Sequential, GCNConv
from torch_geometric.nn import global_mean_pool
from omlt.io.torch_geometric import gnn_with_fixed_graph
import pyomo.environ as pyo
from omlt import OmltBlock


def GCN_Sequential(activation, pooling):
    torch.manual_seed(123)
    return Sequential(
        "x, edge_index",
        [
            (GCNConv(2, 4), "x, edge_index -> x"),
            activation(),
            (GCNConv(4, 4), "x, edge_index -> x"),
            activation(),
            Linear(4, 4),
            (pooling, "x, None -> x"),
            Linear(4, 2),
            activation(),
            Linear(2, 1),
        ],
    )


This model has two types of `Linear` layers: the first linear layer maps in-features to out-features for each node, the last two linear layers map features after pooling. For illustration purposes, we use `load_torch_geometric_sequential` to show the transformed model in OMLT (this step is not needed for later formulation):

In [3]:
from omlt.io.torch_geometric import load_torch_geometric_sequential

# define a GCN sequential model
nn = GCN_Sequential(ReLU, global_mean_pool)
# number of nodes
N = 3
# adjacency matrix
A = np.array([[1, 1, 0], [1, 1, 1], [0, 1, 1]])

# load the model into OMLT
net = load_torch_geometric_sequential(nn, N, A)

for layer_id, layer in enumerate(net.layers):
    print(f"{layer_id}\t{layer}\t{layer.activation}")

0	InputLayer(input_size=[6], output_size=[6])	linear
1	GNNLayer(input_size=[6], output_size=[12])	relu
2	GNNLayer(input_size=[12], output_size=[12])	relu
3	DenseLayer(input_size=[12], output_size=[12])	linear
4	DenseLayer(input_size=[12], output_size=[4])	linear
5	DenseLayer(input_size=[4], output_size=[2])	relu
6	DenseLayer(input_size=[2], output_size=[1])	linear


Two GCN layers are rewritten into two `GNNLayer` in OMLT given $N$ and $A$. The first linear layer is expanded since it maps features of each node. The pooling layer is equivalently transformed into a `DenseLayer`. The last two linear layers are the same as before since features of each node are pooled.

Besides giving $N$ and $A$, one needs to define bounds for inputs:

In [4]:
# define a GCN sequential model
nn1 = GCN_Sequential(ReLU, global_mean_pool)
# number of nodes
N = 3
# adjacency matrix
A = np.array([[1, 1, 0], [1, 1, 1], [0, 1, 1]])

# size of inputs = number of nodes x number of input features
input_size = [6]
# define lower and upper bounds for each input
input_bounds = {}
for i in range(input_size[0]):
    input_bounds[(i)] = (-1.0, 1.0)

After having these information, the last step is to create an `OmltBlock` and build formulation in this block using `gnn_with_fixed_graph`:

In [5]:
# create pyomo model
m1 = pyo.ConcreteModel()

# create an OMLT block for the neural network and build its formulation
m1.nn = OmltBlock()

# build formulation in block m.nn
gnn_with_fixed_graph(m1.nn, nn1, N, A, scaled_input_bounds=input_bounds)

# set the objective as the single output of the model
m1.obj = pyo.Objective(expr=m1.nn.outputs[0])

# solve the optimization problem
status = pyo.SolverFactory("cbc").solve(m1, tee=True)

Welcome to the CBC MILP Solver 
Version: 2.10.10 
Build Date: Aug  1 2023 

command line - /rds/general/user/sz421/home/anaconda3/envs/OMLT_test/bin/cbc -printingOptions all -import /var/tmp/pbs.8152010.pbs/tmpo87uiyi0.pyomo.lp -stat=1 -solve -solu /var/tmp/pbs.8152010.pbs/tmpo87uiyi0.pyomo.soln (default strategy 1)
Option for printingOptions changed from normal to all
Presolve 172 (-222) rows, 111 (-75) columns and 608 (-267) elements
Statistics for presolved model
Original problem has 26 integers (26 of which binary)
Presolved problem has 25 integers (25 of which binary)
==== 110 zero objective 2 different
1 variables have objective of -0.0421598
110 variables have objective of 0
==== absolute objective values 2 different
110 variables have objective of 0
1 variables have objective of 0.0421598
==== for integers 25 zero objective 1 different
25 variables have objective of 0
==== for integers absolute objective values 1 different
25 variables have objective of 0
===== end objective co

We can evaluate the solution in original model to verify it:

In [6]:
X = []
edges = []
for u in range(N):
    for v in range(N):
        if u != v and pyo.value(m1.nn.A[u, v]):
            edges.append((u, v))
for i in range(6):
    X.append(pyo.value(m1.nn.inputs[i]))
X = np.array(X).reshape(3, 2)
edges = np.transpose(np.array(edges)).reshape(2, -1)
nn.eval()
print(nn1(torch.tensor(X).float(), torch.tensor(edges)).detach().numpy())

[[0.31796885]]


## Example 2: Optimizing a GNN with Non-Fixed Graph

Since GCN is not supported when the input graph is not fixed, we define a SAGE in `torch_geometric` as follows:

In [7]:
import numpy as np
import torch
from torch.nn import Linear, ReLU, Sigmoid
from torch_geometric.nn import Sequential, SAGEConv
from torch_geometric.nn import global_add_pool
from omlt.io.torch_geometric import gnn_with_non_fixed_graph

import pyomo.environ as pyo
from omlt import OmltBlock


def SAGE_Sequential(activation, pooling):
    torch.manual_seed(123)
    return Sequential(
        "x, edge_index",
        [
            (SAGEConv(2, 4, aggr="sum"), "x, edge_index -> x"),
            activation(),
            (SAGEConv(4, 4, aggr="sum"), "x, edge_index -> x"),
            activation(),
            Linear(4, 4),
            (pooling, "x, None -> x"),
            Linear(4, 2),
            activation(),
            Linear(2, 1),
        ],
    )

We follow the same procedure as in Example 1 except that $A$ is no longer needed for `gnn_with_non_fixed_graph`:

In [8]:
# define a GAGE sequential model
nn2 = SAGE_Sequential(ReLU, global_add_pool)
# number of nodes
N = 3

# size of inputs = number of nodes x number of input features
input_size = [6]
# define lower and upper bounds for each input
input_bounds = {}
for i in range(input_size[0]):
    input_bounds[(i)] = (-1.0, 1.0)

# create pyomo model
m2 = pyo.ConcreteModel()

# create an OMLT block for the neural network and build its formulation
m2.nn = OmltBlock()

# build formulation in block m.nn
gnn_with_non_fixed_graph(m2.nn, nn2, N, scaled_input_bounds=input_bounds)

# set the objective as the single output of the model
m2.obj = pyo.Objective(expr=m2.nn.outputs[0])

# solve the optimization problem
status = pyo.SolverFactory("cbc").solve(m2, tee=True)

Welcome to the CBC MILP Solver 
Version: 2.10.10 
Build Date: Aug  1 2023 

command line - /rds/general/user/sz421/home/anaconda3/envs/OMLT_test/bin/cbc -printingOptions all -import /var/tmp/pbs.8152010.pbs/tmpnzmuzmeh.pyomo.lp -stat=1 -solve -solu /var/tmp/pbs.8152010.pbs/tmpnzmuzmeh.pyomo.soln (default strategy 1)
Option for printingOptions changed from normal to all
Presolve 260 (-137) rows, 141 (-51) columns and 876 (-197) elements
Statistics for presolved model
Original problem has 32 integers (32 of which binary)
Presolved problem has 29 integers (29 of which binary)
==== 139 zero objective 3 different
139 variables have objective of 0
1 variables have objective of 0.203177
1 variables have objective of 0.686721
==== absolute objective values 3 different
139 variables have objective of 0
1 variables have objective of 0.203177
1 variables have objective of 0.686721
==== for integers 29 zero objective 1 different
29 variables have objective of 0
==== for integers absolute objective

For smooth activation function like Sigmoid, a smooth optimization solvers (such as Ipopt) is needed:

In [10]:
# define a GAGE sequential model
nn3 = SAGE_Sequential(Sigmoid, global_add_pool)
# number of nodes
N = 3

# size of inputs = number of nodes x number of input features
input_size = [6]
# define lower and upper bounds for each input
input_bounds = {}
for i in range(input_size[0]):
    input_bounds[(i)] = (-1.0, 1.0)

# create pyomo model
m3 = pyo.ConcreteModel()

# create an OMLT block for the neural network and build its formulation
m3.nn = OmltBlock()

# build formulation in block m.nn
gnn_with_non_fixed_graph(m3.nn, nn3, N, scaled_input_bounds=input_bounds)

# set the objective as the single output of the model
m3.obj = pyo.Objective(expr=m3.nn.outputs[0])

# solve the optimization problem
status = pyo.SolverFactory("ipopt").solve(m3, tee=True)


Ipopt 3.14.12: 
==> Warning: Treating 6 binary and 0 integer variables as continuous.


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

This is Ipopt version 3.14.12, running with linear solver MUMPS 5.2.1.

Number of nonzeros in equality constraint Jacobian...:      449
Number of nonzeros in inequality constraint Jacobian.:      468
Number of nonzeros in Lagrangian Hessian.............:       26

Total number of variables............................:      166
                     variables with only lower bounds:        0
                variables with lower and upper bounds:      164
                     variables with only upper bounds:        0
Total